In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
class Model (nn.Module):
    def __init__(self, iLayers=146, h1=72, h2=36, h3=16, h4=8, oLayer=1):
        super().__init__()
        self.fc1 = nn.Linear(iLayers, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, h4)
        self.out = nn.Linear(h4, oLayer)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.out(x)

        return x

In [3]:
torch.manual_seed(16)
model = Model()

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [6]:
stats_24 = pd.read_csv('Data Set/21-24Stats.csv')
matchups_24 = pd.read_csv('Data Set/24Matches.csv')

In [7]:
y = matchups_24['SCORE_DIF'].values.reshape(-1,1)
x = matchups_24.merge(stats_24, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_24, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1', 'SCORE_DIF'], axis=1)

scaler = StandardScaler()
x = scaler.fit_transform(x)
    
# Remove NaNs and Infs from the data
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))
y = torch.nan_to_num(torch.tensor(y, dtype=torch.float32))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)

In [9]:
criterion = nn.MSELoss()
# if error doesn't go down after a bunch of iterations (epochs), lower our learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

In [10]:
epochs = 600
losses = []
for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model(x_train) # Get predicted results

  loss = criterion(y_pred, y_train)
  loss = loss / len(x_train) 

  # print every 10 epoch
  if i % 10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  # Do some back propagation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

model.eval()
with torch.no_grad():
    test_pred = model(x_test)
    test_loss = criterion(test_pred, y_test) / len(x_test)
    print(f'Test Loss: {test_loss:.4f}')

Epoch: 0 and loss: 1.985657811164856
Epoch: 10 and loss: 1.9852544069290161
Epoch: 20 and loss: 1.9848746061325073
Epoch: 30 and loss: 1.9844902753829956
Epoch: 40 and loss: 1.9840620756149292
Epoch: 50 and loss: 1.983490228652954
Epoch: 60 and loss: 1.9827656745910645
Epoch: 70 and loss: 1.9818917512893677
Epoch: 80 and loss: 1.9807755947113037
Epoch: 90 and loss: 1.979353666305542
Epoch: 100 and loss: 1.9776556491851807
Epoch: 110 and loss: 1.9756227731704712
Epoch: 120 and loss: 1.9731968641281128
Epoch: 130 and loss: 1.9702810049057007
Epoch: 140 and loss: 1.9667255878448486
Epoch: 150 and loss: 1.9623934030532837
Epoch: 160 and loss: 1.9570508003234863
Epoch: 170 and loss: 1.950446367263794
Epoch: 180 and loss: 1.9423702955245972
Epoch: 190 and loss: 1.9326366186141968
Epoch: 200 and loss: 1.920955777168274
Epoch: 210 and loss: 1.9070416688919067
Epoch: 220 and loss: 1.890618920326233
Epoch: 230 and loss: 1.8714314699172974
Epoch: 240 and loss: 1.8491531610488892
Epoch: 250 and lo

In [11]:
stats_25 = pd.read_csv('Data Set/22-25Stats.csv')
matchups_25 = pd.read_csv('Data Set/25Matches.csv')

In [12]:
y = matchups_25['SCORE_DIF'].values.reshape(-1,1)
x = matchups_25.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1', 'SCORE_DIF'], axis=1)

scaler = StandardScaler()
x = scaler.fit_transform(x)
    
# Remove NaNs and Infs from the data
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))
y = torch.nan_to_num(torch.tensor(y, dtype=torch.float32))

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)

In [14]:
epochs = 200
losses = []
for i in range(epochs):
  # Go forward and get a prediction
  y_pred = model(x_train) # Get predicted results

  loss = criterion(y_pred, y_train)
  loss = loss / len(x_train) 

  # print every 10 epoch
  if i % 10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  # Do some back propagation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

model.eval()
with torch.no_grad():
    test_pred = model(x_test)
    test_loss = criterion(test_pred, y_test) / len(x_test)
    print(f'Test Loss: {test_loss:.4f}')

Epoch: 0 and loss: 1.4653065204620361
Epoch: 10 and loss: 1.3763387203216553
Epoch: 20 and loss: 1.2868996858596802
Epoch: 30 and loss: 1.2317677736282349
Epoch: 40 and loss: 1.193682312965393
Epoch: 50 and loss: 1.162406086921692
Epoch: 60 and loss: 1.1351068019866943
Epoch: 70 and loss: 1.1102097034454346
Epoch: 80 and loss: 1.0867199897766113
Epoch: 90 and loss: 1.0639920234680176
Epoch: 100 and loss: 1.0416439771652222
Epoch: 110 and loss: 1.0195294618606567
Epoch: 120 and loss: 0.9973545074462891
Epoch: 130 and loss: 0.9748269319534302
Epoch: 140 and loss: 0.9520037174224854
Epoch: 150 and loss: 0.9291054606437683
Epoch: 160 and loss: 0.9052437543869019
Epoch: 170 and loss: 0.8805640339851379
Epoch: 180 and loss: 0.8558115363121033
Epoch: 190 and loss: 0.8318691849708557
Test Loss: 2.9849


In [15]:
x = pd.read_csv('Data Set/R64.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [16]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 16.0 Points
Match 2: Team 2 | + 8.0 Points
Match 3: Team 2 | + 2.0 Points
Match 4: Team 2 | + 3.0 Points
Match 5: Team 2 | + 5.0 Points
Match 6: Team 1 | + 3.0 Points
Match 7: Team 2 | + 2.0 Points
Match 8: Team 1 | + 2.0 Points
Match 9: Team 1 | + 9.0 Points
Match 10: Team 2 | + 9.0 Points
Match 11: Team 2 | + 5.0 Points
Match 12: Team 2 | + 3.0 Points
Match 13: Team 2 | + 4.0 Points
Match 14: Team 1 | + 3.0 Points
Match 15: Team 2 | + 4.0 Points
Match 16: Team 1 | + 5.0 Points
Match 17: Team 1 | + 20.0 Points
Match 18: Team 2 | + 6.0 Points
Match 19: Team 2 | + 4.0 Points
Match 20: Team 1 | + 3.0 Points
Match 21: Team 2 | + 5.0 Points
Match 22: Team 1 | + 9.0 Points
Match 23: Team 2 | + 4.0 Points
Match 24: Team 1 | + 6.0 Points
Match 25: Team 1 | + 17.0 Points
Match 26: Team 2 | + 5.0 Points
Match 27: Team 2 | + 3.0 Points
Match 28: Team 2 | + 3.0 Points
Match 29: Team 2 | + 7.0 Points
Match 30: Team 1 | + 1.0 Points
Match 31: Team 2 | + 4.0 Points
Match 32: Team

In [17]:
x = pd.read_csv('Data Set/R32.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [18]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 8.0 Points
Match 2: Team 2 | + 3.0 Points
Match 3: Team 2 | + 3.0 Points
Match 4: Team 1 | + 1.0 Points
Match 5: Team 1 | + 8.0 Points
Match 6: Team 1 | + 1.0 Points
Match 7: Team 2 | + 3.0 Points
Match 8: Team 2 | + 3.0 Points
Match 9: Team 1 | + 12.0 Points
Match 10: Team 2 | + 5.0 Points
Match 11: Team 2 | + 3.0 Points
Match 12: Team 2 | + 5.0 Points
Match 13: Team 1 | + 7.0 Points
Match 14: Team 1 | + 3.0 Points
Match 15: Team 2 | + 5.0 Points
Match 16: Team 2 | + 5.0 Points


In [19]:
x = pd.read_csv('Data Set/S16.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [20]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 11.0 Points
Match 2: Team 2 | + 1.0 Points
Match 3: Team 1 | + 1.0 Points
Match 4: Team 2 | + 4.0 Points
Match 5: Team 1 | + 2.0 Points
Match 6: Team 2 | + 4.0 Points
Match 7: Team 2 | + 1.0 Points
Match 8: Team 2 | + 6.0 Points


In [21]:
x = pd.read_csv('Data Set/E8.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [22]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 14.0 Points
Match 2: Team 2 | + 3.0 Points
Match 3: Team 2 | + 3.0 Points
Match 4: Team 2 | + 5.0 Points


In [23]:
x = pd.read_csv('Data Set/F4.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [24]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 1 | + 2.0 Points
Match 2: Team 2 | + 5.0 Points


In [25]:
x = pd.read_csv('Data Set/Champ.csv')
x = x.merge(stats_25, left_on='TEAM_0_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.merge(stats_25, left_on='TEAM_1_ID', right_on='TEAM_ID', suffixes=('_0', '_1'))
x = x.drop(['MATCH_ID', 'TEAM_0', 'TEAM_1', 'TEAM_0_ID', 'TEAM_1_ID', 'TEAM_ID_0', 'TEAM_ID_1'], axis=1)

x = scaler.fit_transform(x)
x = torch.nan_to_num(torch.tensor(x, dtype=torch.float32))

In [26]:
model.eval()
with torch.no_grad():
    test_pred = model(x)
    pred_winners = ['Team 1' if score > 0 else 'Team 2' for score in test_pred]
    for i, winner in enumerate(pred_winners):
        print(f"Match {i+1}: {winner} | +{np.ceil(abs(test_pred[i].item())): } Points")

Match 1: Team 2 | + 1.0 Points
